## functions

In [1]:
import pandas as pd
import numpy as np
import os
import gc

import addict
import lightgbm as lgb

def save_dataframe(path, dataframe):
    np.save(path + ".data", dataframe.values)
    np.save(path + ".header", dataframe.columns)


def load_dataframe(path):
    data = np.load(path + ".data.npy")
    header = np.load(path + ".header.npy")
    return pd.DataFrame(data=data, columns=header)


def save_dataframe32(path, dataframe, keep=[]):
    col64 = [col_ for col_ in dataframe.columns if col_ in keep]
    col32 = [col_ for col_ in dataframe.columns if col_ not in keep]
    dataframe64 = dataframe[col64]
    dataframe32 = dataframe[col32]
    np.save(path + ".data64", dataframe64.values)
    np.save(path + ".header64", col64)
    np.save(path + ".data32", dataframe32.values.astype(np.float32))
    np.save(path + ".header32", col32)


def load_dataframe32(path, nrows=None):
    path_data32 = path + ".data32.npy"
    path_header32 = path + ".header32.npy"
    path_data64 = path + ".data64.npy"
    path_header64 = path + ".header64.npy"
    result = pd.DataFrame()
    if os.path.exists(path_data32):
        data32 = np.load(path_data32)
        header32 = np.load(path_header32)
        df32 = pd.DataFrame(data=data32, columns=header32)
        result = pd.concat([result, df32], axis=1)
    if os.path.exists(path_data64):
        data64 = np.load(path_data64)
        header64 = np.load(path_header64)
        df64 = pd.DataFrame(data=data64, columns=header64)
        result = pd.concat([result, df64], axis=1)
    if nrows and nrows > 0:
        return result.head(nrows)
    return result

In [2]:
from sklearn.metrics import roc_auc_score
import xgboost as xgb

In [3]:
import glob

In [7]:
from sklearn.linear_model import LogisticRegression

## load oof & pred

### load lgb_info

In [4]:
lgb_info = pd.DataFrame([
    ("oof-024-first.csv       ", 0.79453, 0.002204),
    ("oof-031-lgb-seed-0.csv  ", 0.79426, 0.002599),
    ("oof-031-lgb-seed-1.csv  ", 0.79426, 0.002599),
    ("oof-031-lgb-seed-10.csv ", 0.79446, 0.002332),
    ("oof-031-lgb-seed-11.csv ", 0.79442, 0.002256),
    ("oof-031-lgb-seed-12.csv ", 0.79457, 0.002379),
    ("oof-031-lgb-seed-13.csv ", 0.79451, 0.002476),
    ("oof-031-lgb-seed-14.csv ", 0.79454, 0.002633),
    ("oof-031-lgb-seed-15.csv ", 0.79453, 0.002396),
    ("oof-031-lgb-seed-16.csv ", 0.79414, 0.002331),
    ("oof-031-lgb-seed-17.csv ", 0.79441, 0.002233),
    ("oof-031-lgb-seed-18.csv ", 0.79458, 0.002444),
    ("oof-031-lgb-seed-19.csv ", 0.79447, 0.002104),
    ("oof-031-lgb-seed-2.csv  ", 0.79426, 0.002599),
    ("oof-031-lgb-seed-20.csv ", 0.7943, 0.002261),
    ("oof-031-lgb-seed-21.csv ", 0.79471, 0.002242),
    ("oof-031-lgb-seed-22.csv ", 0.79425, 0.002295),
    ("oof-031-lgb-seed-23.csv ", 0.79419, 0.002687),
    ("oof-031-lgb-seed-24.csv ", 0.79452, 0.002525),
    ("oof-031-lgb-seed-25.csv ", 0.79439, 0.002363),
    ("oof-031-lgb-seed-26.csv ", 0.7945, 0.002492),
    ("oof-031-lgb-seed-27.csv ", 0.79446, 0.002419),
    ("oof-031-lgb-seed-28.csv ", 0.79442, 0.00233),
    ("oof-031-lgb-seed-29.csv ", 0.79433, 0.002703),
    ("oof-031-lgb-seed-3.csv  ", 0.79447, 0.002513),
    ("oof-031-lgb-seed-4.csv  ", 0.79448, 0.002551),
    ("oof-031-lgb-seed-5.csv  ", 0.79462, 0.002398),
    ("oof-031-lgb-seed-6.csv  ", 0.79412, 0.002712),
    ("oof-031-lgb-seed-7.csv  ", 0.79418, 0.002622),
    ("oof-031-lgb-seed-8.csv  ", 0.79461, 0.002116),
    ("oof-031-lgb-seed-9.csv  ", 0.7943, 0.002533)
], columns=["file", "auc", "std"])

lgb_info["file"] = lgb_info.file.apply(lambda x: './oof-result\\' + x.strip())
lgb_info.head()

,file,auc,std
0,./oof-result\oof-024-first.csv,0.79453,0.002204
1,./oof-result\oof-031-lgb-seed-0.csv,0.79426,0.002599
2,./oof-result\oof-031-lgb-seed-1.csv,0.79426,0.002599
3,./oof-result\oof-031-lgb-seed-10.csv,0.79446,0.002332
4,./oof-result\oof-031-lgb-seed-11.csv,0.79442,0.002256


### load oof & test

In [5]:
oof_df = pd.DataFrame({fn: pd.read_csv(fn).oof_pred
    for fn in glob.glob("./oof-result/*.csv")
})

test_df = pd.DataFrame({fn: pd.read_csv(fn.replace("oof-result", "result").replace("oof", "submission")).TARGET
    for fn in glob.glob("./oof-result/*.csv")
})

### cal lgb_avg

In [10]:
def X2rank(X):
    return np.array([calculate_rank(col) for col in X.T]).T

def calculate_rank(predictions):
    rank = (1 + pd.Series(predictions).rank().values) / (predictions.shape[0] + 1)
    return rank

In [11]:
oof_df["lgb_avg"] = X2rank(oof_df[list(lgb_info[lgb_info.auc >= 0.7945].file.values)].values).mean(axis=1)
test_df["lgb_avg"] = test_df[list(lgb_info[lgb_info.auc >= 0.7945].file.values)].values.mean(axis=1)

## stacking

### result 035

In [12]:
# nrows = None  
nrows = None
C = .5
# nrows = 1000

# features = list(np.load('./neptune-features/features_246008_1174_0.npy'))
train_app = load_dataframe32("./bindata/application_train")
auc_valid_stack2 = []
pred_test_stack2 = []

features = list(oof_df.columns)
features = [
    './oof-result\\oof-024-first.csv',
    './oof-result\\oof-024-keras01.csv',
    './oof-result\\oof-028-lgb.csv',
    './oof-result\\oof-030-keras01.csv',
    './oof-result\\oof-031-lgb-seed-10.csv',
    './oof-result\\oof-031-lgb-seed-12.csv',
    './oof-result\\oof-031-lgb-seed-13.csv',
    './oof-result\\oof-031-lgb-seed-15.csv',
    './oof-result\\oof-031-lgb-seed-16.csv',
    './oof-result\\oof-031-lgb-seed-18.csv',
    './oof-result\\oof-031-lgb-seed-21.csv',
    './oof-result\\oof-031-lgb-seed-24.csv',
    './oof-result\\oof-031-lgb-seed-28.csv',
    './oof-result\\oof-031-lgb-seed-5.csv',
    './oof-result\\oof-031-lgb-seed-8.csv',
    './oof-result\\oof-035-xgb-0.csv',
    './oof-result\\oof-035-xgb-1.csv',
    'lgb_avg'
]

# features = features0

for i in range(5):
    train_idx_fn = "./neptune-features/train_idx_{}.npy".format(i)
    valid_idx_fn = "./neptune-features/valid_idx_{}.npy".format(i)

    train_offset, valid_offset, test_offset = i * 4, i * 4 + 1, i * 4 + 3
    
    train_data_fn = "./neptune-features/data_246009_1174_{}".format(train_offset)
    valid_data_fn = "./neptune-features/data_61502_1174_{}".format(valid_offset)
    test_data_fn = "./neptune-features/data_48744_1174_{}".format(test_offset)
    
    if i == 0:
        train_data_fn = train_data_fn.replace("246009", "246008")
        valid_data_fn = valid_data_fn.replace("61502", "61503")
    
    gc.collect()

    train_idx = np.load(train_idx_fn)
    valid_idx = np.load(valid_idx_fn)
    
    if nrows:
        train_idx, valid_idx = train_idx[:nrows].copy(), valid_idx[:nrows].copy()

    gc.collect()
    
    X_train = X2rank(oof_df.loc[train_idx][features].values)
    X_valid = X2rank(oof_df.loc[valid_idx][features].values)
    X_test = test_df[features].values
    
    # X_train = add_std(X_train)
    # X_valid = add_std(X_valid)
    # X_test = add_std(X_test)
    
    y_train = train_app.loc[train_idx].TARGET
    y_valid = train_app.loc[valid_idx].TARGET
    
    print(train_idx_fn, valid_idx_fn)
    print(X_train.shape, y_train.shape)
    print(X_valid.shape, y_valid.shape)
    print(X_test.shape)
    
    estimator = LogisticRegression(C=C)
    estimator.fit(X_train, y_train)
    
    pred_valid = estimator.predict_proba(X_valid)[:, 1]
    auc = roc_auc_score(y_valid, pred_valid)
    auc_valid_stack2.append(auc)
    print("fold-{},auc:{}".format(i, auc))

    pred_test = estimator.predict_proba(X_test)[:, 1]
    pred_test_stack2.append(pred_test)
    
print("kfold-auc, avg:{:.8}, std:{:.2}".format(np.mean(auc_valid_stack2), np.std(auc_valid_stack2)))

./neptune-features/train_idx_0.npy ./neptune-features/valid_idx_0.npy
(246008, 18) (246008,)
(61503, 18) (61503,)
(48744, 18)
fold-0,auc:0.8001245194482286
./neptune-features/train_idx_1.npy ./neptune-features/valid_idx_1.npy
(246009, 18) (246009,)
(61502, 18) (61502,)
(48744, 18)
fold-1,auc:0.7938577203877627
./neptune-features/train_idx_2.npy ./neptune-features/valid_idx_2.npy
(246009, 18) (246009,)
(61502, 18) (61502,)
(48744, 18)
fold-2,auc:0.7939715475829969
./neptune-features/train_idx_3.npy ./neptune-features/valid_idx_3.npy
(246009, 18) (246009,)
(61502, 18) (61502,)
(48744, 18)
fold-3,auc:0.7959402536185475
./neptune-features/train_idx_4.npy ./neptune-features/valid_idx_4.npy
(246009, 18) (246009,)
(61502, 18) (61502,)
(48744, 18)
fold-4,auc:0.7945466007778489
kfold-auc, avg:0.79568813, std:0.0023


### add 036-keras01

In [13]:
# nrows = None  
nrows = None
C = .5
# nrows = 1000

# features = list(np.load('./neptune-features/features_246008_1174_0.npy'))
train_app = load_dataframe32("./bindata/application_train")
auc_valid_stack2 = []
pred_test_stack2 = []

features = list(oof_df.columns)
features = [
    './oof-result\\oof-024-first.csv',
    './oof-result\\oof-024-keras01.csv',
    './oof-result\\oof-028-lgb.csv',
    './oof-result\\oof-030-keras01.csv',
    './oof-result\\oof-036-keras01.csv',
    './oof-result\\oof-031-lgb-seed-10.csv',
    './oof-result\\oof-031-lgb-seed-12.csv',
    './oof-result\\oof-031-lgb-seed-13.csv',
    './oof-result\\oof-031-lgb-seed-15.csv',
    './oof-result\\oof-031-lgb-seed-16.csv',
    './oof-result\\oof-031-lgb-seed-18.csv',
    './oof-result\\oof-031-lgb-seed-21.csv',
    './oof-result\\oof-031-lgb-seed-24.csv',
    './oof-result\\oof-031-lgb-seed-28.csv',
    './oof-result\\oof-031-lgb-seed-5.csv',
    './oof-result\\oof-031-lgb-seed-8.csv',
    './oof-result\\oof-035-xgb-0.csv',
    './oof-result\\oof-035-xgb-1.csv',
    'lgb_avg'
]

# features = features0

for i in range(5):
    train_idx_fn = "./neptune-features/train_idx_{}.npy".format(i)
    valid_idx_fn = "./neptune-features/valid_idx_{}.npy".format(i)

    train_offset, valid_offset, test_offset = i * 4, i * 4 + 1, i * 4 + 3
    
    train_data_fn = "./neptune-features/data_246009_1174_{}".format(train_offset)
    valid_data_fn = "./neptune-features/data_61502_1174_{}".format(valid_offset)
    test_data_fn = "./neptune-features/data_48744_1174_{}".format(test_offset)
    
    if i == 0:
        train_data_fn = train_data_fn.replace("246009", "246008")
        valid_data_fn = valid_data_fn.replace("61502", "61503")
    
    gc.collect()

    train_idx = np.load(train_idx_fn)
    valid_idx = np.load(valid_idx_fn)
    
    if nrows:
        train_idx, valid_idx = train_idx[:nrows].copy(), valid_idx[:nrows].copy()

    gc.collect()
    
    X_train = X2rank(oof_df.loc[train_idx][features].values)
    X_valid = X2rank(oof_df.loc[valid_idx][features].values)
    X_test = test_df[features].values
    
    # X_train = add_std(X_train)
    # X_valid = add_std(X_valid)
    # X_test = add_std(X_test)
    
    y_train = train_app.loc[train_idx].TARGET
    y_valid = train_app.loc[valid_idx].TARGET
    
    print(train_idx_fn, valid_idx_fn)
    print(X_train.shape, y_train.shape)
    print(X_valid.shape, y_valid.shape)
    print(X_test.shape)
    
    estimator = LogisticRegression(C=C)
    estimator.fit(X_train, y_train)
    
    pred_valid = estimator.predict_proba(X_valid)[:, 1]
    auc = roc_auc_score(y_valid, pred_valid)
    auc_valid_stack2.append(auc)
    print("fold-{},auc:{}".format(i, auc))

    pred_test = estimator.predict_proba(X_test)[:, 1]
    pred_test_stack2.append(pred_test)
    
print("kfold-auc, avg:{:.8}, std:{:.2}".format(np.mean(auc_valid_stack2), np.std(auc_valid_stack2)))

./neptune-features/train_idx_0.npy ./neptune-features/valid_idx_0.npy
(246008, 19) (246008,)
(61503, 19) (61503,)
(48744, 19)
fold-0,auc:0.800137974559402
./neptune-features/train_idx_1.npy ./neptune-features/valid_idx_1.npy
(246009, 19) (246009,)
(61502, 19) (61502,)
(48744, 19)
fold-1,auc:0.7939231838498191
./neptune-features/train_idx_2.npy ./neptune-features/valid_idx_2.npy
(246009, 19) (246009,)
(61502, 19) (61502,)
(48744, 19)
fold-2,auc:0.7940336516608175
./neptune-features/train_idx_3.npy ./neptune-features/valid_idx_3.npy
(246009, 19) (246009,)
(61502, 19) (61502,)
(48744, 19)
fold-3,auc:0.7959960272342395
./neptune-features/train_idx_4.npy ./neptune-features/valid_idx_4.npy
(246009, 19) (246009,)
(61502, 19) (61502,)
(48744, 19)
fold-4,auc:0.7945078093303993
kfold-auc, avg:0.79571973, std:0.0023


### add features

In [22]:
def add_top_ratio(X, th):
    new_col = (X > th).sum(axis=1)
    return np.hstack([X, new_col.reshape((X.shape[0], 1)).astype("float") / X.shape[1]])

def add_bottom_ratio(X, th):
    new_col = (X < th).sum(axis=1)
    return np.hstack([X, new_col.reshape((X.shape[0], 1)).astype("float") / X.shape[1]])

In [26]:
X = np.array([
   [1, 1, 3],
   [1, 1, 3],
   [1, 2, 3],
   [1, 2, 1],
   [1, 1, 1],
   [1, 1, 2],
   [1, 1, 2],
   [1, 1, 3]
])

add_top_ratio(X, 1.5)

array([[1.        , 1.        , 3.        , 0.33333333],
       [1.        , 1.        , 3.        , 0.33333333],
       [1.        , 2.        , 3.        , 0.66666667],
       [1.        , 2.        , 1.        , 0.33333333],
       [1.        , 1.        , 1.        , 0.        ],
       [1.        , 1.        , 2.        , 0.33333333],
       [1.        , 1.        , 2.        , 0.33333333],
       [1.        , 1.        , 3.        , 0.33333333]])

In [55]:
# nrows = None  
nrows = None
C = .5
# nrows = 1000

# features = list(np.load('./neptune-features/features_246008_1174_0.npy'))
train_app = load_dataframe32("./bindata/application_train")
auc_valid_stack2 = []
pred_test_stack2 = []

features = list(oof_df.columns)
features = [
    './oof-result\\oof-024-first.csv',
    './oof-result\\oof-024-keras01.csv',
    './oof-result\\oof-028-lgb.csv',
    './oof-result\\oof-030-keras01.csv',
    './oof-result\\oof-036-keras01.csv',
    './oof-result\\oof-031-lgb-seed-10.csv',
    './oof-result\\oof-031-lgb-seed-12.csv',
    './oof-result\\oof-031-lgb-seed-13.csv',
    './oof-result\\oof-031-lgb-seed-15.csv',
    './oof-result\\oof-031-lgb-seed-16.csv',
    './oof-result\\oof-031-lgb-seed-18.csv',
    './oof-result\\oof-031-lgb-seed-21.csv',
    './oof-result\\oof-031-lgb-seed-24.csv',
    './oof-result\\oof-031-lgb-seed-28.csv',
    './oof-result\\oof-031-lgb-seed-5.csv',
    './oof-result\\oof-031-lgb-seed-8.csv',
    './oof-result\\oof-035-xgb-0.csv',
    './oof-result\\oof-035-xgb-1.csv',
    'lgb_avg'
]

# features = features0

for i in range(5):
    train_idx_fn = "./neptune-features/train_idx_{}.npy".format(i)
    valid_idx_fn = "./neptune-features/valid_idx_{}.npy".format(i)

    train_offset, valid_offset, test_offset = i * 4, i * 4 + 1, i * 4 + 3
    
    train_data_fn = "./neptune-features/data_246009_1174_{}".format(train_offset)
    valid_data_fn = "./neptune-features/data_61502_1174_{}".format(valid_offset)
    test_data_fn = "./neptune-features/data_48744_1174_{}".format(test_offset)
    
    if i == 0:
        train_data_fn = train_data_fn.replace("246009", "246008")
        valid_data_fn = valid_data_fn.replace("61502", "61503")
    
    gc.collect()

    train_idx = np.load(train_idx_fn)
    valid_idx = np.load(valid_idx_fn)
    
    if nrows:
        train_idx, valid_idx = train_idx[:nrows].copy(), valid_idx[:nrows].copy()

    gc.collect()
    
    X_train = X2rank(oof_df.loc[train_idx][features].values)
    X_valid = X2rank(oof_df.loc[valid_idx][features].values)
    X_test = test_df[features].values
    
    top_th = .9
    X_train = add_top_ratio(X_train, top_th)
    X_valid = add_top_ratio(X_valid, top_th)
    X_test = add_top_ratio(X_test, top_th)
    
    bottom_th = .32
    X_train = add_bottom_ratio(X_train, bottom_th)
    X_valid = add_bottom_ratio(X_valid, bottom_th)
    X_test = add_bottom_ratio(X_test, bottom_th)
    
    # X_train = add_std(X_train)
    # X_valid = add_std(X_valid)
    # X_test = add_std(X_test)
    
    y_train = train_app.loc[train_idx].TARGET
    y_valid = train_app.loc[valid_idx].TARGET
    
    print(train_idx_fn, valid_idx_fn)
    print(X_train.shape, y_train.shape)
    print(X_valid.shape, y_valid.shape)
    print(X_test.shape)
    
    estimator = LogisticRegression(C=C)
    estimator.fit(X_train, y_train)
    
    pred_valid = estimator.predict_proba(X_valid)[:, 1]
    auc = roc_auc_score(y_valid, pred_valid)
    auc_valid_stack2.append(auc)
    print("fold-{},auc:{}".format(i, auc))

    pred_test = estimator.predict_proba(X_test)[:, 1]
    pred_test_stack2.append(pred_test)
    
    # break
    
print("kfold-auc, avg:{:.8}, std:{:.2}".format(np.mean(auc_valid_stack2), np.std(auc_valid_stack2)))

./neptune-features/train_idx_0.npy ./neptune-features/valid_idx_0.npy
(246008, 21) (246008,)
(61503, 21) (61503,)
(48744, 21)
fold-0,auc:0.8003330612030864
./neptune-features/train_idx_1.npy ./neptune-features/valid_idx_1.npy
(246009, 21) (246009,)
(61502, 21) (61502,)
(48744, 21)
fold-1,auc:0.7939519363314395
./neptune-features/train_idx_2.npy ./neptune-features/valid_idx_2.npy
(246009, 21) (246009,)
(61502, 21) (61502,)
(48744, 21)
fold-2,auc:0.794259140085628
./neptune-features/train_idx_3.npy ./neptune-features/valid_idx_3.npy
(246009, 21) (246009,)
(61502, 21) (61502,)
(48744, 21)
fold-3,auc:0.7960197281709537
./neptune-features/train_idx_4.npy ./neptune-features/valid_idx_4.npy
(246009, 21) (246009,)
(61502, 21) (61502,)
(48744, 21)
fold-4,auc:0.7946879264745857
kfold-auc, avg:0.79585036, std:0.0023


### submission

In [56]:
pred_target = sum([calculate_rank(p_) for p_ in pred_test_stack2]) / 5

pred_sample = pd.read_csv("./result/submission-022-blend08.csv")
pred_sample["TARGET"] = pred_target

pred_sample.to_csv("./result/submission-037-stacking-logit-fs-above-bottom-0.csv", index=False)

### blending

In [58]:
pred_sample = pd.read_csv("./result/submission-022-blend08.csv")
pred_target = pd.read_csv("./result/submission-037-stacking-logit-fs-above-bottom-0.csv").TARGET
bld_param = .25
pred_sample["TARGET"] = pred_target * (1 - bld_param) + pred_sample.TARGET * bld_param
pred_sample.to_csv("./result/submission-037-stacking-logit-fs-above-bottom-0-bld{}.csv".format(str(bld_param).replace(".", "-")), index=False)